In [0]:
import numpy as np
data = np.array([-2, 2, -1, 0])

# x -> (x, x**2)
def transform(x):
  x_hat = []
  for i in x:
    x_hat.append([i, i**2])
  return np.array(x_hat)

label = np.array([1, 1, -1, -1])

In [2]:
from sklearn import svm
# 1. Transformation
x_hat = transform(data)

# 2. Training SVM on transformed data
model = svm.SVC(kernel="linear")
model.fit(x_hat, label)

coeff = abs(model.dual_coef_)
alpha_coeff = np.zeros(len(data))
for idx, c in enumerate(model.support_):
  alpha_coeff[c] = coeff[0][idx]
w0 = model.intercept_
print(alpha_coeff)

[0.1667 0.0555 0.2222 0.    ]


In [3]:
# 3. Classification of new point using calculations
import numpy as np
x_5 = np.array([1])
x_5_hat = transform(x_5)
tot = 0
for i in range(len(alpha_coeff)):
  tot += alpha_coeff[i] * label[i] * np.dot(x_hat[i], x_5_hat.T)
print("Classification of new point through calculation: {}".format(np.sign(tot + w0)))
print("Classification of new point through prediction: {}".format(model.predict(x_5_hat)))

Classification of new point through calculation: [-1.]
Classification of new point through prediction: [-1]


4.a) $k(x_i, x) = \langle \tilde{x}_i, \tilde{x} \rangle \; = \; \langle \Phi({x}_i), \Phi({x}) \rangle \; {=} \; x_ix \: + x_i^2x^2 \; = \; x_ix(1+x_ix)$

In [4]:
# 4.b Classification of new value without projection
a = []
for i in data:
  for j in x_5:
    a.append(i * j * (1 + i * j))
a = np.array(a)

print("Classification of new point: {}".format(np.sign(np.sum([alpha_coeff[i] * label[i] * a[i] for i in range(len(data))]))))

Classification of new point: 1.0


In [5]:
# 5. Define own kernel based on the calculation in #4.a and find the coefficient
def my_kernel(x, x_new):
  a = np.zeros((len(x), len(x_new)))
  for idx_x, val_x in enumerate(x):
    for idx_j, val_j in enumerate(x_new):
      a[idx_x, idx_j] = val_x * val_j * (1 + val_x * val_j)
  return a

model_ = svm.SVC(kernel=my_kernel)
model_.fit(data.reshape(-1, 1), label)

coeff_ = abs(model_.dual_coef_)
alpha_coeff_ = np.zeros(len(data))
for idx, c in enumerate(model_.support_):
  alpha_coeff_[c] = coeff_[0][idx]

print("Coefficient obtained using transformation(linear kernel): {}".format(alpha_coeff))
print("Coefficient obtained without transformation(custom kernel): {}".format(alpha_coeff_))

Coefficient obtained using transformation(linear kernel): [0.1667 0.0555 0.2222 0.    ]
Coefficient obtained without transformation(custom kernel): [0.1667 0.0555 0.2222 0.    ]


In [6]:
# 6. Normalized distance of new point from the hyperplane
distance = np.sum([alpha_coeff[i] * label[i] * my_kernel([data[i]], [x_5]) for i in range(len(data))]) + w0

norm_weight = 0
for i in range(len(data)):
  for j in range(len(data)):
    norm_weight += np.sum([alpha_coeff[i] * alpha_coeff[j] * label[i] * label[j] * my_kernel([data[i]], [data[j]]) for i in range(len(data))]) 

norm_weight = np.sqrt(norm_weight)
print(distance/norm_weight)

[-0.75017498]
